In [1]:
import numpy as np
import pandas as pd

In [5]:
for file in ["train", "test"]:

    #obtain some extra info
    df = pd.read_csv(file+"_features.csv")

    ratios = pd.DataFrame(df.groupby(["pid"]).last()/df.groupby(["pid"]).first()-1).iloc[:,2:]
    ratios.columns = [x+"_ratio" for x in ratios.columns]
    ratios.reset_index(drop=True, inplace=True)

    derivative = pd.DataFrame(df.groupby(["pid"]).last()-df.groupby(["pid"]).first()).iloc[:,2:]
    derivative.columns = [x+"_derivative" for x in derivative.columns]
    derivative.reset_index(drop=True, inplace=True)

    count = pd.DataFrame(df.groupby(["pid"]).count()).iloc[:,2:]
    count.columns = [x+"_count" for x in count.columns]
    count.reset_index(drop=True, inplace=True)


    df = pd.read_csv(file+"_features.csv",index_col="pid")
    #forward fill missing values
    df = df.groupby(["pid"]).ffill().reset_index()
    df.set_index("pid",inplace=True)
    #backward fill missing values
    df = df.groupby(["pid"]).bfill().reset_index()

    #some columns are missing entirely, create new columns to indicate that those values were imputed
    missing = df.groupby(["pid"]).last().isna().astype(int).iloc[:,2:]
    missing.columns = [x+"_missing" for x in missing.columns]

    #group every 12 rows together with new column names like "Temp_1", "Temp_2", etc.
    columns = ["data_"+str(x) for x in range(1,409)]
    data = np.array(df)[:, 3:]
    data = data.reshape(data.shape[0]//12,34*12)
    data = pd.DataFrame(data, columns=columns)

    #append
    df1 = df.iloc[::12,:3]
    df2 = data
    df3 = missing
    df1.reset_index(drop=True, inplace=True)
    df2.reset_index(drop=True, inplace=True)
    df3.reset_index(drop=True, inplace=True)
    df = pd.concat([df1,df2,df3,ratios,derivative,count],axis=1)
    #fill nan
    df = df.fillna(0)
    #replace inf with 0
    df = df.replace([np.inf, -np.inf], 0)

    df.to_csv(file+'_features_improved.csv', index=False, float_format='%.3f',na_rep='nan')

In [3]:
#other predictions
# df = pd.read_csv("submit.csv")
# df.iloc[:,:11]

In [4]:
# file="test"

# #obtain some extra info
# df = pd.read_csv(file+"_features.csv")

# ratios = pd.DataFrame(df.groupby(["pid"]).last()/df.groupby(["pid"]).first()-1).iloc[:,2:]
# ratios.columns = [x+"_ratio" for x in ratios.columns]
# ratios.reset_index(drop=True, inplace=True)

# derivative = pd.DataFrame(df.groupby(["pid"]).last()-df.groupby(["pid"]).first()).iloc[:,2:]
# derivative.columns = [x+"_derivative" for x in derivative.columns]
# derivative.reset_index(drop=True, inplace=True)

# count = pd.DataFrame(df.groupby(["pid"]).count()).iloc[:,2:]
# count.columns = [x+"_count" for x in count.columns]
# count.reset_index(drop=True, inplace=True)


# df = pd.read_csv(file+"_features.csv",index_col="pid")
# #forward fill missing values
# df = df.groupby(["pid"]).ffill().reset_index()
# df.set_index("pid",inplace=True)
# #backward fill missing values
# df = df.groupby(["pid"]).bfill().reset_index()

# #some columns are missing entirely, create new columns to indicate that those values were imputed
# missing = df.groupby(["pid"]).last().isna().astype(int).iloc[:,2:]
# missing.columns = [x+"_missing" for x in missing.columns]

# #group every 12 rows together with new column names like "Temp_1", "Temp_2", etc.
# columns = ["data_"+str(x) for x in range(1,409)]
# data = np.array(df)[:, 3:].reshape(151969//12,34*12)
# data = pd.DataFrame(data, columns=columns)

# #append
# df1 = df.iloc[::12,:3]
# df2 = data
# df3 = missing
# df1.reset_index(drop=True, inplace=True)
# df2.reset_index(drop=True, inplace=True)
# df3.reset_index(drop=True, inplace=True)
# df = pd.concat([df1,df2,df3,ratios,derivative,count],axis=1)
# #fill nan
# df = df.fillna(0)
# #replace inf with 0
# df = df.replace([np.inf, -np.inf], 0)

# df.to_csv(file+'_features_improved.csv', index=False, float_format='%.3f',na_rep='nan')